In [146]:
def find_next_shortest_path(my_list, orders_list):
    shortest_distance = {}
    for i in range(len(orders_list)):
        if orders_list[i][0] != 'home':
            shortest_distance[orders_list[i][0]] = ((my_list[1] - orders_list[i][1]) ** 2 + (my_list[2] - orders_list[i][2]) ** 2) ** 0.5
    for i in range(len(orders_list)):
        if orders_list[i][0] == min(shortest_distance, key=shortest_distance.get):
            return [orders_list[i], shortest_distance[min(shortest_distance, key=shortest_distance.get)]]

def manipulate_orders(orders):
    my_list = []
    for i in range(len(orders)):
        my_list.append([orders[i][0], float(orders[i][1]), float(orders[i][2])])
    return my_list

def manipulate_airports(airport):
    my_list = []
    for i in range(len(airport)):
        my_list.append([airport[i][0], float(airport[i][1]), float(airport[i][2]), int(airport[i][3])])
    return my_list

def find_furthest_path(orders_list):
    shortest_distance = {}
    list_to_return = []
    for i in range(len(orders_list)):
        if orders_list[i][0] != 'home':
            shortest_distance[orders_list[i][0]] = ((orders_list[0][1] - orders_list[i][1]) ** 2 + (orders_list[0][2] - orders_list[i][2]) ** 2) ** 0.5
    for i in range(len(orders_list)):
        if max(shortest_distance, key=shortest_distance.get) == orders_list[i][0]:
            return [orders_list[i], shortest_distance[max(shortest_distance, key=shortest_distance.get)]]

def find_nearest_airport_from_home(home_list,airports):
    airport = {}
    for i in range(len(airports)):
        airport[airports[i][0]] =  distance_calculation(home_list ,airports[i])
    for i in range(len(airports)):
        if min(airport, key=airport.get) == airports[i][0]:
            airports[i].append(airport[airports[i][0]])
            return airports[i]
        
def find_orders_from_home(orders_list, airport_dist):
    shortest_distance = {}
    list_to_return = []
    for i in range(len(orders_list)):
        if orders_list[i][0] != 'home':
            shortest_distance[orders_list[i][0]] = [orders_list[i][0], orders_list[i][1], orders_list[i][2], ((orders_list[0][1] - orders_list[i][1]) ** 2 + (orders_list[0][2] - orders_list[i][2]) ** 2) ** 0.5]
    for key in shortest_distance:
        if shortest_distance[key][3] <= airport_dist:
            list_to_return.append(shortest_distance[key])
    return list_to_return
        
def nearest_airport_from_furthest_dist(furthest_dist, airports):
    airport = {}
    for i in range(len(airports)):
        airport[airports[i][0]] =  distance_calculation(furthest_dist ,airports[i])
    for i in range(len(airports)):
        if min(airport, key=airport.get) == airports[i][0]:
            airports[i].append(airport[airports[i][0]])
            return airports[i]
        
def find_all_orders_from_airport(orders_list, nearest_airport):
    shortest_distance = {}
    list_to_return = []
    for i in range(len(orders_list)):
        if orders_list[i][0] != 'home':
            shortest_distance[orders_list[i][0]] = [orders_list[i][0], orders_list[i][1], orders_list[i][2], distance_calculation(nearest_airport ,orders_list[i])]
    for key in shortest_distance:
        if shortest_distance[key][3] <= nearest_airport[4]:
            list_to_return.append(shortest_distance[key])
    return list_to_return

def remove_from_list(string, orders_list):
    for orders in orders_list:
        if string in orders:
            orders_list.remove(orders)
            return orders_list

def remove_airport(string, airports):
    for airport in airports:
        if string in airport:
            airports.remove(airport)
            return airports        

def schedule_q2(orders, airports, truck_speed, plane_speed, number_trucks):
    tmp_list = []
    list_to_return = []
    clustered_orders = []
    orders.insert(0, ['home', 0, 0])
    
    orders_list = manipulate_orders(orders)
    airports = manipulate_airports(airports)
    threshold = (len(orders) // number_trucks)

    home_airport = '' 
    
    nearest_airport_from_home = find_nearest_airport_from_home(orders_list[0], airports)
    home_airport = nearest_airport_from_home
    capacity_home = nearest_airport_from_home[3]
    if nearest_airport_from_home[3] == 0:
        while (nearest_airport_from_home[3] == 0):
            airports = remove_airport(nearest_airport_from_home[0], airports)
            nearest_airport_from_home = find_nearest_airport_from_home(orders_list[0], airports)
    else:
        capacity_home = capacity_home - 1
        airports = remove_airport(nearest_airport_from_home[0], airports)
        
    last_to_append = find_orders_from_home(orders_list, nearest_airport_from_home[4])

    previous_airport = ''
   
    next_airport = ''
    
    # add airport validation.
    next_path = find_furthest_path(orders_list)
    nearest_airport = nearest_airport_from_furthest_dist(next_path[0], airports)
    orders_list = remove_from_list(next_path[0][0], orders_list)
#     tmp_list.append(nearest_airport_from_home[0])
#     tmp_list.append(nearest_airport[0])
    tmp_list.append(next_path[0])
    previous_airport = nearest_airport
    
    while (len(orders_list) > 1):
        next_path = find_next_shortest_path(next_path[0], orders_list)
        truck_travel_dist = distance_calculation(tmp_list[-1], next_path[0])
        
        if truck_travel_dist < 55:
            tmp_list.append(next_path[0])
            orders_list = remove_from_list(next_path[0][0], orders_list)
        else:
            #find the nearest airport
            #calculate plane distance
            nearest_airport = nearest_airport_from_furthest_dist(next_path[0], airports)
            nearest_airport_from_prev = nearest_airport_from_furthest_dist(tmp_list[-1], airports)
            
            order_to_airport = distance_calculation(next_path[0], nearest_airport)
            airport_flight = (distance_calculation(nearest_airport,  nearest_airport_from_prev)) / plane_speed
            airport_to_prev = distance_calculation(nearest_airport_from_prev, tmp_list[-1])
            
            total_distance_cal = airport_to_prev + airport_flight + order_to_airport
            if total_distance_cal < truck_travel_dist and nearest_airport_from_prev[3] > 0:
                tmp_list.append(nearest_airport_from_prev)
                tmp_list.append(nearest_airport)
                tmp_list.append(next_path[0])
                nearest_airport[3] -= 1
                orders_list = remove_from_list(next_path[0][0], orders_list)

    
    tmp_new_list = [] 
    final_list = []
    

    counter = 2
    for string in tmp_list:
        if counter > threshold:
            final_list.append(tmp_new_list)
            tmp_new_list = []
            counter = 0
        if string[0] != 'A':
            tmp_new_list.append(string)
            counter += 1
        else:
            tmp_new_list.append(string)
    
    final_list.append(tmp_new_list)
    
    cost = 0
    cost_list = []
    cnt = 0
    
    for lists in final_list:
        #if home -> first order by truck is nearer than home -> plane -> first order
        if lists[0][0][0] == 'A' and home_airport[3] > 0 and lists[0][0] != home_airport[0]:
            if lists[1][0] == home_airport[0]:
                final_list[cnt] = lists[2:]
            else:
                lists[0] = home_airport
                home_airport[3] -= 1
        else:
            travel_by_truck_from_home = distance_calculation(orders_list[0], lists[0])
            next_home_airport = nearest_airport_from_furthest_dist(lists[0], airports)
            first_order_to_airplane = distance_calculation(next_home_airport, lists[0])
            home_airport_distance = (distance_calculation(home_airport, next_home_airport)) / plane_speed
            total_home_dist = next_home_airport[-1] + first_order_to_airplane + home_airport_distance
            if total_home_dist < travel_by_truck_from_home and home_airport[3] > 0:
                cost_diff = travel_by_truck_from_home - total_home_dist
                cost_list.append([cost_diff, home_airport[0], next_home_airport[0], cnt])

            
        travel_by_truck = distance_calculation(lists[-1], orders_list[0])

        next_airport = nearest_airport_from_furthest_dist(lists[-1], airports)
        last_order_to_airplane = distance_calculation(lists[-1], next_airport)
        airport_distance = distance_calculation(next_airport, home_airport) / plane_speed
        airport_to_home = distance_calculation(home_airport, orders_list[0])
        new_capacity = next_airport[3]
        total_dist = last_order_to_airplane + airport_distance + airport_to_home
        if total_dist < travel_by_truck and new_capacity > 0:
            lists.append(next_airport[0])
            lists.append(home_airport[0])
            new_capacity = new_capacity - 1
            next_airport[3] = new_capacity
        cnt += 1

    
    for i in range(home_airport[3]):
        if len(cost_list) == 0:
            break
        else:
            data = max(cost_list, key=lambda x: x[0])
            final_list[data[3]].insert(0, data[1])
            final_list[data[3]].insert(1, data[2])
            cost_list.remove(data)

    
    tmp1_list = []
    final_list2 = []
    counter1 = 0

    for to_append in final_list:
        for j in range(len(to_append)):
            if to_append[j][0] == 'A':
                tmp1_list.append(to_append[j])
                counter1 += 1
            else:
                tmp1_list.append(to_append[j][0])
                counter1 += 1

        final_list2.append(tmp1_list)
        tmp1_list = []
        counter1 = 0
    print(final_list2)
    return final_list2
                




In [147]:
from utility import *
import copy

def scoring_q2(truck_paths, location_dict, truck_speed, plane_speed):
	cost_list = [] # list of time costs for each truck. Each element represents the cost for 1 truck
	for path in truck_paths:
		cost_list.append(0) # to be modified later. each item in cost_list corresponds to one path in truck_paths 
		take_plane = -1  # keeps track of 1st or 2nd airport (-1 means next airport is 1st airport)
		for e, order in enumerate(path):
			
			if e == 0:  # back to company
				print(location_dict[path[e]])                  
				cost_list[-1] += (distance_calculation(location_dict[path[e]], [0, 0, 0]) / truck_speed)
				print("back to company")  
				print((distance_calculation(location_dict[path[e]], [0, 0, 0]) / truck_speed))
			if e == len(path) - 1: # last order in path
				cost_list[-1] += (distance_calculation(location_dict[path[e]], [0, 0, 0]) / truck_speed)
				print("last order in path")
				print((distance_calculation(location_dict[path[e]], [0, 0, 0]) / truck_speed))
			else:
				if order[0] == "A": # take plane
					if take_plane == -1: # 1st airport. take plane to 2nd airport (use plane speed)
						cost_list[-1] += (distance_calculation(location_dict[path[e]], location_dict[path[e + 1]]) / plane_speed)
						print('take plane 1')
						print((distance_calculation(location_dict[path[e]], location_dict[path[e + 1]]) / plane_speed))
					else:  # 2nd (destination) airport. take truck to next location (use truck speed)                      
						cost_list[-1] += (distance_calculation(location_dict[path[e]], location_dict[path[e + 1]]) / truck_speed)
						print('take plane 2')
						print((distance_calculation(location_dict[path[e]], location_dict[path[e + 1]]) / truck_speed))

					take_plane *= -1 # toggle

				if order[0] == "O": # drive to order 
					cost_list[-1] += (distance_calculation(location_dict[path[e]], location_dict[path[e + 1]]) / truck_speed) 
					print('orders')
					print((distance_calculation(location_dict[path[e]], location_dict[path[e + 1]]) / truck_speed))
	return cost_list


def legality_checking(truck_paths, order_list, airport_list, number_trucks):
	check_trucks(truck_paths, number_trucks)
	checking_all_order(truck_paths, order_list)
	legality_checking_airline_completeness(truck_paths)
	legality_checking_airline_capacity(truck_paths, airport_list)

# replace these parameters with different csv file locations
order_csv = "./dataset/orders_001.csv"
parameter_csv = "./dataset/parameters_001.csv"
airport_csv = "./dataset/airports_001.csv"

number_trucks, truck_speed, plane_speed = parameter_reader(parameter_csv)
orders = list_reader(order_csv)
airports = list_reader(airport_csv)
location_dict = location_dict_generation([orders, airports])

# these will b used on line 32. just in case your schedule_q2 modifies orders and airports. 
orders_clone = copy.deepcopy(orders)     # clone
airports_clone = copy.deepcopy(airports) # clone 

# call your function
truck_paths = schedule_q2(orders, airports, truck_speed, plane_speed, number_trucks) 

# check returned value
legality_checking(truck_paths, orders_clone, airports_clone, number_trucks)

# print score 
print("Score for q2 is : ", scoring_q2(truck_paths, location_dict, truck_speed, plane_speed)) # print your score (smaller is better)








[['A00003', 'A00006', 'O00048', 'O00027', 'O00193', 'O00279', 'O00165', 'O00098', 'O00176', 'O00302', 'O00305', 'O00146', 'O00052', 'A00006', 'A00003'], ['A00003', 'A00006', 'O00219', 'O00205', 'O00000', 'O00082', 'O00296', 'O00037', 'O00057', 'O00170', 'O00265', 'O00152', 'O00042', 'O00036', 'O00311', 'A00005', 'A00003'], ['A00003', 'A00005', 'O00055', 'O00169', 'O00208', 'O00011', 'O00003', 'O00237', 'O00102', 'O00147', 'O00135', 'O00113', 'O00155', 'O00079', 'O00253'], ['O00263', 'O00050', 'O00030', 'O00004', 'O00199', 'O00132', 'O00294', 'O00112', 'O00184', 'O00204', 'O00013', 'O00231', 'O00245'], ['O00122', 'O00087', 'O00240', 'O00269', 'O00143', 'O00156', 'O00175', 'O00271', 'O00078', 'O00092', 'O00126', 'O00107', 'O00129'], ['O00117', 'O00217', 'O00153', 'O00239', 'O00018', 'O00283', 'O00238', 'O00233', 'O00137', 'O00298', 'O00059', 'O00065', 'O00179'], ['O00124', 'O00026', 'O00086', 'O00045', 'O00020', 'O00076', 'O00229', 'O00236', 'O00259', 'O00012', 'O00218', 'O00257', 'O0017

In [ ]:
#     list_to_return.append([nearest_airport_from_home[0], nearest_airport[0]])
#     tmp_list = (find_all_orders_from_airport(orders_list, nearest_airport))
#     for i in range(len(tmp_list)):
#         if tmp_list[i][0] == furthest_dist[0]:
#             to_insert = tmp_list.pop(i)
#             break
#     tmp_list.insert(0, to_insert)
#     #find nearest neighbours around the list.
#     list_to_return.append(tmp_list)
#     for i in range(len(tmp_list)):
#         orders_list = remove_from_list(tmp_list[i][0], orders_list)
#     previous_airport = nearest_airport
#     airports = remove_airport(nearest_airport[0], airports)
    
#     while (len(orders_list) > 1):
#         furthest_dist = find_furthest_path(orders_list)
#         nearest_airport = nearest_airport_from_furthest_dist(furthest_dist, airports)
#         tmp_list = (find_all_orders_from_airport(orders_list, nearest_airport))
#         for i in range(len(tmp_list)):
#             if tmp_list[i][0] == furthest_dist[0]:
#                 to_insert = tmp_list.pop(i)
#                 break
#         tmp_list.insert(0, to_insert)
#         #find nearest neighbour around the list
#         list_to_return.append(tmp_list)
#         for i in range(len(tmp_list)):
#             orders_list = remove_from_list(tmp_list[i][0], orders_list)
#         airports = remove_airport(nearest_airport[0], airports)
#         if len(orders_list) > 1:
#             list_to_return.append([previous_airport[0], nearest_airport[0]])
#         previous_airport = nearest_airport

#     list_to_return.append([previous_airport[0], nearest_airport_from_home[0]])
#     list_to_return.append(last_to_append)
    
#     tmp_new_list = []
#     final_list = []
    
#     orders_list = manipulate_orders(orders)
    
#     count = 0
#     for my_list in list_to_return:
#         if 'A' not in my_list[0][0]:
#             list_to_return[count] = two_opt(my_list)
#         count += 1
    
#     counter = 0

#     for my_list in list_to_return:
#         for string in my_list:
#             if counter > threshold:
#                 final_list.append(tmp_new_list)
#                 tmp_new_list = []
#                 counter = 0

#             if string[0] != 'A':
#                 tmp_new_list.append(string[0])
#                 counter += 1
#             else:
#                 tmp_new_list.append(string)
    
#     final_list.append(tmp_new_list)
    
#     return final_list

    

In [2]:
print(2 % 4)

2
